In [1]:
from pyspark.sql.session import SparkSession
from pyspark.sql.context import SQLContext

In [2]:
#spark session for connection to a postgreSQL database
spark = SparkSession.builder.appName("masniputaing_database")\
                            .config("spark.driver.extraClassPath","../helpers/postgresql-42.7.3.jar")\
                            .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0")\
                            .getOrCreate()
#sql context
sqlcontext = SQLContext(spark)

c:\Users\juanf\anaconda3\envs\dataenv\lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
#defining the jdbc connection
jdbc_url = "jdbc:postgresql://localhost:5432/postgres"

In [4]:
#reading from SQL
sf_fire_calls_df = spark.read.format("jdbc")\
                      .option("url",f"{jdbc_url}")\
                      .option("driver", "org.postgresql.Driver")\
                      .option("user","postgres")\
                      .option("password","Felipe1998")\
                      .option("dbtable","pyspark_tables.fire_service_calls")\
                      .load()

sf_fire_calls_df.createOrReplaceTempView("fire_service_calls_view")

How many types of calls were made to the fire department ?

In [13]:
type_of_calls_query = """
    SELECT call_type, count(call_type) as number_of_calls, COUNT(call_type) OVER() AS distinc_type_of_calls_count
    FROM fire_service_calls_view
    GROUP BY call_type
    ORDER BY number_of_calls DESC;
"""

In [14]:
type_of_calls_df = spark.sql(type_of_calls_query)

In [15]:
type_of_calls_df.show()

+--------------------+---------------+---------------------------+
|           call_type|number_of_calls|distinc_type_of_calls_count|
+--------------------+---------------+---------------------------+
|    Medical Incident|            600|                         18|
|Structure Fire / ...|            147|                         18|
|              Alarms|            112|                         18|
|   Traffic Collision|             72|                         18|
|Citizen Assist / ...|             18|                         18|
|               Other|             13|                         18|
|        Vehicle Fire|              8|                         18|
|Odor (Strange / U...|              7|                         18|
|Gas Leak (Natural...|              6|                         18|
|        Outside Fire|              4|                         18|
|Smoke Investigati...|              4|                         18|
|        Water Rescue|              2|                        

What zip codes accounted for most common calls?

In [10]:
zip_call_types_query = """

    SELECT zipcode_of_incident, call_type, COUNT(*) AS count_
    FROM fire_service_calls_view
    GROUP BY zipcode_of_incident, call_type
    ORDER BY count_ DESC;

"""

In [11]:
zip_call_types_df = spark.sql(zip_call_types_query)

In [12]:
zip_call_types_df.show(10)

+-------------------+----------------+------+
|zipcode_of_incident|       call_type|count_|
+-------------------+----------------+------+
|              94102|Medical Incident|    80|
|              94103|Medical Incident|    68|
|              94110|Medical Incident|    62|
|              94109|Medical Incident|    56|
|              94124|Medical Incident|    33|
|              94112|Medical Incident|    33|
|              94133|Medical Incident|    29|
|              94115|Medical Incident|    23|
|              94107|Medical Incident|    23|
|              94122|Medical Incident|    20|
+-------------------+----------------+------+
only showing top 10 rows



What San Francisco neighborhoods are in zip codes 94102 and 94103?

In [24]:
neighborhoods_on_zip_code = """

    SELECT zipcode_of_incident, neighborhoods_analysis_boundaries
    FROM fire_service_calls_view
    WHERE zipcode_of_incident IN (94102, 94103)
    ORDER BY zipcode_of_incident;

"""

In [25]:
neighborhoods_on_zip_code_df = spark.sql(neighborhoods_on_zip_code)

In [26]:
neighborhoods_on_zip_code_df.show(10)

+-------------------+---------------------------------+
|zipcode_of_incident|neighborhoods_analysis_boundaries|
+-------------------+---------------------------------+
|              94102|                     Hayes Valley|
|              94102|                       Tenderloin|
|              94102|                       Tenderloin|
|              94102|                       Tenderloin|
|              94102|                     Hayes Valley|
|              94102|                       Tenderloin|
|              94102|                       Tenderloin|
|              94102|                     Hayes Valley|
|              94102|                       Tenderloin|
|              94102|                       Tenderloin|
+-------------------+---------------------------------+
only showing top 10 rows



How many distinct years of data is in the data set?

In [27]:
distinct_year = """ 

    SELECT DISTINCT EXTRACT(YEAR FROM call_date) AS year_calls
    FROM fire_service_calls_view ;

"""

In [28]:
distinct_year_df = spark.sql(distinct_year)

In [29]:
distinct_year_df.show()

+----------+
|year_calls|
+----------+
|      2006|
+----------+

